# 🧠 Latent Dirichlet Allocation (Manual - Gibbs Sampling)

Notebook ini melakukan pemodelan topik (Topic Modeling) secara **manual tanpa library sklearn/gensim**, menggunakan konsep **Gibbs Sampling**.

Dataset: `df_preprocessing.csv`
- `tokenize_indo` → hasil tokenisasi berita
- `Kategori Berita` → label kategori (tidak dipakai pada LDA unsupervised)


In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict, Counter

# Load dataset
df = pd.read_csv("datasets/df_preprocessing.csv")

# Ambil kolom yang sudah dipreprocessing
documents = df["tokenize_indo"].astype(str).tolist()
labels = df["Kategori Berita"].tolist()

print("Jumlah dokumen:", len(documents))
df.head()

Jumlah dokumen: 1200


,Isi Berita,lwr_indo,clean_sw_indo,clean_stb_indo,clean_typo_indo,stemming_indo,tokenize_indo,Kategori Berita
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,kompas.com - menteri pemuda dan olahraga erick...,kompas.com - menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick thohi...,kompas com menteri pemuda olahraga erick thohi...,"['kompas', 'com', 'menteri', 'pemuda', 'olahra...",BOLA
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men...","kompas.com - manajer chelsea, enzo maresca men...","kompas.com - manajer chelsea, enzo maresca men...",kompas com manajer chelsea enzo maresca meng...,kompas com manajer chelsea enzo maresca mengan...,kompas com manajer chelsea enzo maresca anggap...,"['kompas', 'com', 'manajer', 'chelsea', 'enzo'...",BOLA
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...","kompas.com - pelatih liverpool, arne slot, men...","kompas.com - pelatih liverpool, arne slot, men...",kompas com pelatih liverpool arne slot menga...,kompas com pelatih liverpool arne slot mengaku...,kompas com latih liverpool arne slot aku cryst...,"['kompas', 'com', 'latih', 'liverpool', 'arne'...",BOLA
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,kompas.com - hasil terbaru pekan kelima liga i...,kompas.com - hasil terbaru pekan liga italia 2...,kompas com hasil terbaru pekan liga italia ...,kompas com hasil terbaru pekan liga italia per...,kompas com hasil baru pekan liga italia beda n...,"['kompas', 'com', 'hasil', 'baru', 'pekan', 'l...",BOLA
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,kompas.com - menteri pemuda dan olahraga repub...,kompas.com - menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik in...,kompas com menteri pemuda olahraga republik in...,"['kompas', 'com', 'menteri', 'pemuda', 'olahra...",BOLA


In [2]:
# Ubah string tokenisasi menjadi list kata dan bersihkan
def clean_tokens(text):
    text = text.replace('[', '').replace(']', '').replace("'", "").replace(",", "")
    return [w.strip() for w in text.split() if len(w) > 2]

docs = [clean_tokens(doc) for doc in documents]

# Tampilkan contoh
for i in range(3):
    print(f"Dokumen {i+1}:", docs[i][:15])

Dokumen 1: ['kompas', 'com', 'menteri', 'pemuda', 'olahraga', 'erick', 'thohir', 'langsung', 'gebrak', 'minggu', 'minggu', 'jabat', 'resmi', 'cabut', 'permenpora']
Dokumen 2: ['kompas', 'com', 'manajer', 'chelsea', 'enzo', 'maresca', 'anggap', 'sulit', 'henti', 'liverpool', 'tahan', 'gelar', 'juara', 'liga', 'inggris']
Dokumen 3: ['kompas', 'com', 'latih', 'liverpool', 'arne', 'slot', 'aku', 'crystal', 'palace', 'milik', 'kuat', 'patut', 'hitung', 'liverpool', 'racik']


In [3]:
# Tentukan parameter LDA
K = 5          # jumlah topik
alpha = 0.1    # hyperparameter topik per dokumen
beta = 0.01    # hyperparameter kata per topik
iterations = 20

# Bangun vocabulary
vocab = list(set([word for doc in docs for word in doc]))
V = len(vocab)
word2id = {w: i for i, w in enumerate(vocab)}

print("Jumlah kosakata (V):", V)
print("Contoh kosakata:", vocab[:15])

Jumlah kosakata (V): 17566
Contoh kosakata: ['resale', 'asiakapolres', 'serie', 'kelembaban', 'istirahat', 'wisnu', 'doodle', 'lpsk', 'perkosa', 'rengkuh', 'tualang', 'kuas', 'islamic', 'mitsuhashiokamura', 'tempe']


In [4]:
# Inisialisasi penetapan topik acak
doc_topics = []
for doc in docs:
    topics = [random.randint(0, K-1) for _ in doc]
    doc_topics.append(topics)

n_dk = np.zeros((len(docs), K))  # jumlah kata per dokumen per topik
n_kv = np.zeros((K, V))          # jumlah kata per topik
n_k = np.zeros(K)                # total kata per topik

for d_idx, doc in enumerate(docs):
    for w, z in zip(doc, doc_topics[d_idx]):
        w_id = word2id[w]
        n_dk[d_idx][z] += 1
        n_kv[z][w_id] += 1
        n_k[z] += 1

print("Inisialisasi topik selesai ✅")

Inisialisasi topik selesai ✅


In [5]:
# Fungsi untuk sampling ulang topik
def sample_topic(d_idx, w_id):
    probs = np.zeros(K)
    for k in range(K):
        p_w_t = (n_kv[k][w_id] + beta) / (n_k[k] + V * beta)
        p_t_d = (n_dk[d_idx][k] + alpha)
        probs[k] = p_w_t * p_t_d
    probs /= np.sum(probs)
    return np.random.choice(np.arange(K), p=probs)

In [6]:
# Iterasi Gibbs Sampling
for it in range(iterations):
    for d_idx, doc in enumerate(docs):
        for i, w in enumerate(doc):
            old_topic = doc_topics[d_idx][i]
            w_id = word2id[w]

            # Kurangi count lama
            n_dk[d_idx][old_topic] -= 1
            n_kv[old_topic][w_id] -= 1
            n_k[old_topic] -= 1

            # Sampling topik baru
            new_topic = sample_topic(d_idx, w_id)

            # Update count baru
            doc_topics[d_idx][i] = new_topic
            n_dk[d_idx][new_topic] += 1
            n_kv[new_topic][w_id] += 1
            n_k[new_topic] += 1

    print(f"Iterasi ke-{it+1} selesai")

Iterasi ke-1 selesai
Iterasi ke-2 selesai
Iterasi ke-3 selesai
Iterasi ke-4 selesai
Iterasi ke-5 selesai
Iterasi ke-6 selesai
Iterasi ke-7 selesai
Iterasi ke-8 selesai
Iterasi ke-9 selesai
Iterasi ke-10 selesai
Iterasi ke-11 selesai
Iterasi ke-12 selesai
Iterasi ke-13 selesai
Iterasi ke-14 selesai
Iterasi ke-15 selesai
Iterasi ke-16 selesai
Iterasi ke-17 selesai
Iterasi ke-18 selesai
Iterasi ke-19 selesai
Iterasi ke-20 selesai


In [7]:
# Menampilkan kata dominan per topik
num_words = 10
for k in range(K):
    top_indices = np.argsort(n_kv[k])[::-1][:num_words]
    top_words = [vocab[i] for i in top_indices]
    print(f"\nTopik {k+1}:")
    print(", ".join(top_words))


Topik 1:
jalan, juga, baca, jakarta, kompas, prabowo, com, indonesia, masyarakat, marquez

Topik 2:
main, hasil, tim, laga, gol, juga, baca, menang, emas, menit

Topik 3:
baca, juga, menteri, korban, perintah, laku, uang, orang, rumah, itu

Topik 4:
mbg, baca, mobil, juga, kompas, com, makan, motor, kendara, racun

Topik 5:
indonesia, juga, baca, jakarta, resmi, persen, kompas, kerja, harga, usaha


In [8]:
# Distribusi topik per dokumen
topic_dist = (n_dk + alpha) / np.sum(n_dk + alpha, axis=1, keepdims=True)

for i, dist in enumerate(topic_dist):
    dominant_topic = np.argmax(dist)
    print(f"Dokumen {i+1}: Topik dominan = {dominant_topic+1} | Distribusi = {np.round(dist, 3)}")

Dokumen 1: Topik dominan = 1 | Distribusi = [0.446 0.108 0.359 0.    0.087]
Dokumen 2: Topik dominan = 2 | Distribusi = [0.174 0.517 0.009 0.031 0.269]
Dokumen 3: Topik dominan = 2 | Distribusi = [0.015 0.61  0.    0.    0.375]
Dokumen 4: Topik dominan = 2 | Distribusi = [0.02  0.882 0.    0.    0.097]
Dokumen 5: Topik dominan = 1 | Distribusi = [0.365 0.34  0.188 0.036 0.071]
Dokumen 6: Topik dominan = 2 | Distribusi = [0.368 0.61  0.005 0.    0.018]
Dokumen 7: Topik dominan = 2 | Distribusi = [0.033 0.701 0.001 0.001 0.265]
Dokumen 8: Topik dominan = 5 | Distribusi = [0.039 0.35  0.07  0.    0.54 ]
Dokumen 9: Topik dominan = 1 | Distribusi = [0.489 0.288 0.206 0.016 0.001]
Dokumen 10: Topik dominan = 2 | Distribusi = [0.024 0.572 0.    0.    0.403]
Dokumen 11: Topik dominan = 2 | Distribusi = [0.    0.463 0.07  0.146 0.32 ]
Dokumen 12: Topik dominan = 2 | Distribusi = [0.084 0.802 0.001 0.007 0.107]
Dokumen 13: Topik dominan = 2 | Distribusi = [0.02  0.614 0.004 0.    0.361]
Dokumen 